# INNIO Fleet Analysis

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
import numpy as np
import matplotlib.pyplot as plt
import bokeh
from collections import namedtuple
from pprint import (pprint as pp, pformat as pf)
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import HTML, display
import ipywidgets as widgets
import arrow

import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(3600)

### Filter Engines from installed fleet 

In [ ]:
def sfun(x):
    return all([
            ("Forsa Hartmoor" in str(x['IB Site Name'])), 
            #("Landshut" in str(x['IB Site Name'])), 
            #("M4" in str(x['Engine ID'])), 
            (x['OperationalCondition'] != 'Decommissioned')
        ])

In [ ]:
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

In [ ]:
ddl = pd.DataFrame(fleet['serialNumber'] + ' - ' + fleet['IB Site Name'] + ' ' + fleet['Engine ID'])[0].to_list()
ddl = [(m, i) for i, m in enumerate(ddl)]
dl=widgets.Dropdown(options = ddl,value=0,description='Engine:',layout={'width':'max-content'},disabled=False)
display(dl)

In [ ]:
e=dmyplant2.Engine.from_fleet(mp,motor:=fleet.iloc[dl.value])
pd.DataFrame.from_dict(e.dash, orient='index').T

In [ ]:
from dfsm import msgFSM
#fsm = msgFSM(e, skip_days=7) #funktioniert nicht
#fsm = msgFSM(e, p_from=motor['Commissioning Date'], p_to=arrow.now() )
fsm = msgFSM(e, p_from="2022-01-01", p_to=arrow.now(), frompickle=True)
#fsm = msgFSM(e)
fsm.run1(enforce=False) # run Finite State Machine
fsm.store()

In [ ]:
# Struktur zur Aufnahme mehrerer motoren
fsm_data = []
fsm_data.append({'engine':e, 'fsm':fsm})

In [ ]:
res = fsm_data[0]
res['result'] = pd.DataFrame(fsm_data[0]['fsm']._starts)
fsm.summary(res)

In [ ]:
pd.DataFrame(fsm._starts)[['index'] +['startpreparation','starter','hochlauf','idle','synchronize','loadramp']].round(2)

In [ ]:
rmodes = ['???','OFF','MANUAL','AUTO']; mw = [] ; modes = []
for mm in rmodes:
    mw.append(widgets.Checkbox(value=False, description='Mode: ' + mm))
display(widgets.VBox(mw))

In [ ]:
rsucc = [True,False]; sw = []; success=[]
for rs in rsucc:
    sw.append(widgets.Checkbox(value=False, description='Success: ' + str(rs)))
display(widgets.VBox(sw))

In [ ]:
rdf = res['result']
modes = [rmodes[i] for i,v in enumerate(mw) if v.value]
modes = rmodes if not modes else modes
success = [rsucc[i] for i,v in enumerate(sw) if v.value]
success = rsucc if not success else success

rda = rdf[:].reset_index(drop='index')
rda['count_alarms'] = rda.apply(lambda x: len(x['alarms']), axis=1)
rda['count_warnings'] = rda.apply(lambda x: len(x['warnings']), axis=1)
rda = rda[((rdf['mode'].isin(modes)) & (rdf['success'].isin(success)))].reset_index(drop='index')
print(modes, success)

In [ ]:
# special filters can be added like ... 
#rda = rda[((rda['loadramp'] < 80.0) & (rda['startpreparation'] < 300.0))].reset_index(drop='index')

# rda = rda[((rda['count_warnings'] == 0) & (rda['count_alarms'] == 0))].reset_index(drop='index')
# display the filterd & selected data
rda[fsm.filters['filter_content'] + fsm.filters['filter_alarms_and_warnings']].round(2)

In [ ]:
rda = fsm.run2(rda)
fsm.store()

In [ ]:
rdb = rda[(rda['mode'] == 'AUTO') & rda['success'] & (rda['maxload'] > 4000.0)]
rdb[fsm.filters['run2filter_content'] + fsm.filters['filter_alarms_and_warnings']].round(2)

In [ ]:
rdb[['startpreparation','hochlauf','idle','synchronize','loadramp','maxload','ramprate','cumstarttime']].hist(bins=30,figsize=(20,20));

In [ ]:
rdb[['startpreparation','hochlauf','idle','synchronize','loadramp','maxload','ramprate','cumstarttime']].describe().round(2)

In [ ]:
startversuch = rda.iloc[65];
von_dt=pd.to_datetime(startversuch['starttime']); von=int(von_dt.timestamp())
bis_dt=pd.to_datetime(startversuch['endtime']); bis=int(bis_dt.timestamp())

ftitle = f"{fsm._e} ----- Start {startversuch.name} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
print(f"von: {von_dt.strftime('%d.%m.%Y %H:%M:%S')} = {von} bis: {bis_dt.strftime('%d.%m.%Y %H:%M:%S')} = {bis}")

In [ ]:
vset = fsm._data_spec + ['Hyd_PressCrankCase','Hyd_PressOilDif','Hyd_PressOil','Hyd_TempOil']
data = fsm.get_cycle_data(startversuch, max_length=None, min_length=None, cycletime=1, silent=False, p_data=vset)

In [ ]:
dset = [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue'},
    {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange'},
    #{'col':['Hyd_PressOilDif'],'ylim': [0, 2], 'color':'black'},
    {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown'},
    {'col':['Hyd_TempOil'],'ylim': [0, 110], 'color':'#2171b5'}
]
fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(16,10), style='line', line_width=0)

sv_lines = [v for v in startversuch[fsm.filters['vertical_lines_times']] if v==v]
start = startversuch['starttime']; lines=list(np.cumsum(sv_lines))
dmyplant2.add_dbokeh_lines(start,lines,fig,line_color='red', line_dash='solid', line_alpha=0.4)

bokeh.plotting.show(fig)

In [ ]:
fig, ax, axes = dmyplant2.chart(data, dset, figsize=(14,8), title=ftitle)
sv_lines = [v for v in startversuch[fsm.filters['vertical_lines_times']] if v==v]
start = startversuch['starttime']; lines=list(np.cumsum(sv_lines))
dmyplant2.add_lines(start, lines, ax, color='gray', linestyle="-.")

In [ ]:
dset = [
    {'col':['Various_Values_SpeedAct','Various_Values_SpeedAct_left', 'Various_Values_SpeedAct_right'], 'ylim':(0,10000)}
]
pl = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='left')
pr = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='right')
fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(14,8), style='line', line_width=0)
dmyplant2.add_dbokeh_lines(pl.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
dmyplant2.add_dbokeh_lines(pr.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
bokeh.plotting.show(fig)

In [ ]:
dset = [
    {'col':['Power_PowerAct','Power_PowerAct_left','Power_PowerAct_right'], 'ylim':(0,40000)}
]
pl = fsm.detect_edge(data, 'Power_PowerAct', kind='left')
pr = fsm.detect_edge(data, 'Power_PowerAct', kind='right')
fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(14,8), style='line', line_width=0)
dmyplant2.add_dbokeh_lines(pl.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
dmyplant2.add_dbokeh_lines(pr.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
bokeh.plotting.show(fig)


In [ ]:
dset = [
    {'col':['Power_PowerAct','Various_Values_SpeedAct'], 'ylim':(0,30000)},
    {'col':['Power_PowerAct_left','Power_PowerAct_right','Various_Values_SpeedAct_left', 'Various_Values_SpeedAct_right'], 'ylim':(0,30000)}
    ]

pl = fsm.detect_edge(data, 'Power_PowerAct', kind='left')
pr = fsm.detect_edge(data, 'Power_PowerAct', kind='right')
sl = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='left')
sr = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='right')
#fig, ax, axes = dmyplant2.chart(data, dset, figsize=(14,8), title=ftitle)
summary = pd.DataFrame(startversuch[fsm.filters['filter_times']], dtype=np.float64).round(2).T
display(HTML('<h3>'+ summary.to_html(escape=False, index=False) + '</h3>'))

fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(14,8), style='line', line_width=0)

sv_lines = [v for v in startversuch[fsm.filters['vertical_lines_times']] if v==v]
start = startversuch['starttime']; lines=list(np.cumsum(sv_lines))
dmyplant2.add_dbokeh_lines(start, lines, fig,line_color='red', line_dash='solid', line_alpha=0.4)

dmyplant2.add_dbokeh_lines(sl.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
dmyplant2.add_dbokeh_lines(sr.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
dmyplant2.add_dbokeh_lines(pl.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
dmyplant2.add_dbokeh_lines(pr.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
bokeh.plotting.show(fig)


In [ ]:
# dset = [
#     {'col':['Power_PowerAct','Various_Values_SpeedAct'], '_ylim':(0,10000)}
# ]
dset = [
    {'col':['Power_PowerAct'], '_ylim':(0,5000), 'color':'red'},
    {'col':['Various_Values_SpeedAct'],'_ylim': [0, 2500], 'color':'blue'},
    {'col':['Hyd_PressCrankCase'],'_ylim': [-100, 100], 'color':'orange'},
    {'col':['Hyd_PressOilDif'],'_ylim': [0, 2], 'color':'black'},
    {'col':['Hyd_TempOil'],'_ylim': [60, 110], 'color':'#2171b5'}
]
for ii,startversuch in rda.iterrows():
        if ii < 0:
                continue
        if ii > 10:
                break
        data = fsm.get_cycle_data(startversuch, max_length=None, min_length=None, cycletime=1, p_data=fsm._data_spec + ['Hyd_PressCrankCase','Hyd_PressOilDif','Hyd_TempOil'])

        pl = fsm.detect_edge(data, 'Power_PowerAct', kind='left')
        pr = fsm.detect_edge(data, 'Power_PowerAct', kind='right')
        sl = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='left')
        sr = fsm.detect_edge(data, 'Various_Values_SpeedAct', kind='right')

        ftitle = f"{fsm._e} ----- Start {startversuch['index']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"

        fig = dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(16,10), style='line', line_width=0)

        ml = (data.iloc[-1]['time'] - data.iloc[0]['time']) // 1000
        sv_lines = [v for v in startversuch[fsm.filters['vertical_lines_times']]]
        start = startversuch['starttime']; 
        #lines=list(np.cumsum(sv_lines))
        nsv_lines = [v for v in sv_lines if ((v==v) and (v <= ml)) ]
        lines=list(np.cumsum(nsv_lines))
        dmyplant2.add_dbokeh_lines(start, lines, fig,line_color='red', line_dash='solid', line_alpha=0.4)

        dmyplant2.add_dbokeh_lines(sl.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
        dmyplant2.add_dbokeh_lines(sr.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
        dmyplant2.add_dbokeh_lines(pl.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
        dmyplant2.add_dbokeh_lines(pr.loc, [], fig,line_color='red', line_dash='solid', line_alpha=0.4)
        bokeh.plotting.show(fig) 
                
        svdf = pd.DataFrame(sv_lines, index=fsm.filters['vertical_lines_times'], columns=['FSM']).fillna(0)
        svdf['RUN2'] = svdf['FSM']
        #if svdf.at['hochlauf','FSM'] > 0.0:
        #        svdf.at['hochlauf','RUN2'] = sl.loc.timestamp() - start.timestamp() - np.cumsum(svdf['RUN2'])['starter']
        #        svdf.at['idle','RUN2'] = svdf.at['idle','FSM'] - (svdf.at['hochlauf','RUN2'] - svdf.at['hochlauf','FSM'])
        if svdf.at['loadramp','FSM'] > 0.0:
                svdf.at['loadramp','RUN2'] = pl.loc.timestamp() - start.timestamp() - np.cumsum(svdf['RUN2'])['synchronize']
        with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                svdf = pd.concat([
                        svdf, 
                        pd.DataFrame.from_dict(
                                {       'maxload':['-',pl.val],
                                        'ramp':['-',(pl.val / fsm._e['Power_PowerNominal']) * 100 / svdf.at['loadramp','RUN2']],
                                        'cumstarttime':[np.cumsum(svdf['FSM'])['loadramp'], np.cumsum(svdf['RUN2'])['loadramp']]
                                }, 
                                columns=['FSM','RUN2'],
                                orient='index')]
                        )
        summary = pd.DataFrame(startversuch[fsm.filters['filter_times']], dtype=np.float64).round(2).T
        display(HTML('<h3>'+ summary.to_html(escape=False, index=False) + '</h3>'))

        for i, al in enumerate(startversuch['alarms']):
                print(f"{al['state']:16} {fsm.msgtxt(al['msg'],i)}")

        for i, w in enumerate(startversuch['warnings']):
                print(f"{w['state']:16} {fsm.msgtxt(w['msg'],i)}")

        plt.show();

In [ ]:
rda.describe().round(1).T

In [ ]:
longer_than_300s = rda[fsm.filters['filter_period'] + fsm.filters['filter_content']][rda['cumstarttime'] > 300.0]
longer_than_300s[fsm.filters['filter_content']].round(2)

In [ ]:
longer_than_300s.describe().round(2).T

In [ ]:
load_ramp_less_than_100s = rdb[fsm.filters['filter_period'] + fsm.filters['run2filter_content']][(rdb['loadramp'] < 100.0) & (rdb['maxload'] > 4000.0)]
lfsm = load_ramp_less_than_100s[['starttime']+fsm.filters['run2filter_content']]
lfsm = lfsm.sort_values(by = "index",ascending=True).reset_index(drop='index').round(2)
lfsm

In [ ]:
rdc = rdb[rdb.starttime > '2022-01-01']
dset = [
    {'col':['ramprate'],'ylim':(-1,3)},
    {'col':['cumstarttime'],'ylim':(0,900), 'color':'darkblue'},
    {'col':['synchronize'],'ylim':(0,400)},
    {'col':['startpreparation'],'ylim':(-100,200)},
    {'col':['hochlauf'],'ylim':(0,100), 'color':'black'},
    {'col':['maxload'],'ylim':(500,5500) }
]
ftitle = f"{fsm._e}"
fig = dmyplant2.dbokeh_chart(rdc, dset, x='starttime', figsize=(16,10) ,title=ftitle);
bokeh.plotting.show(fig)

In [ ]:
dset = [
    {'col':['ramprate'],'ylim':(0,1.5)},
    {'col':['synchronize'],'ylim':(0,200)},
    {'col':['cumstarttime'],'ylim':(0,400)},
    {'col':['maxload'],'ylim':(0,5000) }
]
dmyplant2.chart(rdb, dset, x='starttime', figsize=(12,8), title=ftitle);

In [ ]:
rda.reset_index(drop='index').iloc[10];

In [ ]:
for i, r in load_ramp_less_than_100s.reset_index(drop='index').iterrows():
    if i >= 2:
        break
    fsm.plot_cycle(r, max_length=8*60, ylim=(0,2500), cycletime=1, marker=None,figsize=(15,8), title=f"{i:3d} - {fsm._e} {r['starttime'].round('S')}")
    plt.show()


In [ ]:
load_ramp_less_than_100s.describe().round(2).T

In [ ]:
rda[['startpreparation','starter','hochlauf','idle','synchronize','cumstarttime','loadramp','targetoperation']].hist(figsize = (20,12), bins=80, layout=(3,3));
#rda.hist(figsize = (20,12), bins=100, layout=(4,2));

In [ ]:
nalarms = []
ct = 0
ct2 = 0
mini = 0
maxi = 100
for i,c in rdf.iterrows():
    if len(c['alarms']) > 0 and not c['success']:
        ct += 1
        print(f"\nStartversuch: {i}, Success: {c['success']}")
        for a in c['alarms']:
            nalarms.append(a['msg'])
            _txt = f"{ct2} {c['mode']:15} {a['state']:20} {a['msg']['timestamp']} {pd.to_datetime(int(a['msg']['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {a['msg']['name']} {a['msg']['message']}"
            print(_txt)
            if ct2 >= mini and ct2 <= maxi:
                fsm.plot_cycle(c, ylim=(0,2500), cycletime=1, marker=None,figsize=(20,12), title=f"{i:3d} - {fsm._e} {_txt}")
                plt.show()
            ct2 += 1

print(f"""
***********************************
** {ct:3} nicht erfolgreiche Starts **
***********************************
""")

In [ ]:
al = pd.DataFrame(fsm._pareto(nalarms))
fig = plt.figure();
color = 'purple'

if not al.empty:
    al['msg'] = al['msg'] + ' (' + al['name'] + ')'
    al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title='Alarms in not successful Starts');
    plt.plot();

In [ ]:
phases = [['startpreparation','starter','hochlauf','idle','synchronize','loadramp','targetoperation'],'startpreparation','starter','hochlauf','idle','synchronize','loadramp','targetoperation']
fig = plt.figure();
color = 'red'

for phase in phases:
    al = fsm.alarms_pareto(phase)[:30]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
fig = plt.figure();
color = 'orange'

for phase in phases:
    al = fsm.warnings_pareto(phase)[:30]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
mfn = e._fname + '_messages.txt'
fsm.save_messages(mfn)
print(mfn)

In [ ]:
any(fsm._messages['name'] == '9047')

In [ ]:
fsm._target_load_message

In [ ]:
#fsm.store()